In [1]:
import pandas as pd
import numpy as np
import os


# grid search holt winter's exponential smoothing
from math import sqrt
from multiprocessing import cpu_count
from joblib import Parallel
from joblib import delayed
from warnings import catch_warnings
from warnings import filterwarnings
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from sklearn.metrics import mean_squared_error
from numpy import array

In [2]:
train_df, validation_df, test_df = pd.read_csv('data/train_fil_3.csv',index_col=0),pd.read_csv('data/validation_fil_3.csv',index_col=0),pd.read_csv('data/test_fil_3.csv',index_col=0)
for df in [train_df,validation_df,test_df]:
    df.columns = [int(col) for col in df.columns]

In [3]:
#function to calculate MAPE for all observations where y_true is not 0
def mape(y_true, y_predict):
    '''Returns mean percentage error for all predictions where y_true is not 0. Where y_true is 0, the percentage error is 0 as well '''
    return np.mean([np.absolute(y_true[idx] - y_predict[idx])/y_true[idx] * 100 if y_true[idx] != 0 else 0 for idx,_ in enumerate(y_true) ])

def median_pe(y_true, y_predict):
    '''Returns mean percentage error for all predictions where y_true is not 0. Where y_true is 0, the percentage error is 0 as well '''
    return np.median([np.absolute(y_true[idx] - y_predict[idx])/y_true[idx] * 100 if y_true[idx] != 0 else 0 for idx,_ in enumerate(y_true) ])

In [4]:
# one-step Holt Winter’s Exponential Smoothing forecast
def exp_smoothing_forecast(history, config):
	t,d,s,p,b,r = config
	# define model
	history = array(history)
	model = ExponentialSmoothing(history, trend=t, damped=d, seasonal=s, seasonal_periods=p)
	# fit model
	model_fit = model.fit(optimized=True, use_boxcox=b, remove_bias=r)
	# make one step forecast
	yhat = model_fit.predict(len(history), len(history))
	return yhat[0]

# walk-forward validation for univariate data
def walk_forward_validation(train,test, cfg):
	predictions = list()
	# seed history with training dataset
	history = [x for x in train]
	# step over each time-step in the test set
	for i in range(len(test)):
		# fit model and make forecast for history
		yhat = exp_smoothing_forecast(history, cfg)
		# store forecast in list of predictions
		predictions.append(yhat)
		# add actual observation to history for the next loop
		history.append(test[i])
	# estimate prediction error
	error = mape(test, predictions)
	return error
 
# score a model, return None on failure
def score_model(train, test, cfg, debug=False):
	result = None
	# convert config to a key
	key = str(cfg)
	# show all warnings and fail on exception if debugging
	if debug:
		result = walk_forward_validation(train, test, cfg)
	else:
		# one failure during model validation suggests an unstable config
		try:
			# never show warnings when grid searching, too noisy
			with catch_warnings():
				filterwarnings("ignore")
				result = walk_forward_validation(train, test, cfg)
		except:
			error = None
	# check for an interesting result
	if result is not None:
		print(' > Model[%s] %.3f' % (key, result))
	return (key, result)
 
# grid search configs
def grid_search(train, test, cfg_list, parallel=True):
	scores = None
	if parallel:
		# execute configs in parallel
		executor = Parallel(n_jobs=cpu_count(), backend='multiprocessing')
		tasks = (delayed(score_model)(train, test, cfg) for cfg in cfg_list)
		scores = executor(tasks)
	else:
		scores = [score_model(train, test, cfg) for cfg in cfg_list]
	# remove empty results
	scores = [r for r in scores if r[1] != None]
	# sort configs by error, asc
	scores.sort(key=lambda tup: tup[1])
	return scores
 
# create a set of exponential smoothing configs to try
def exp_smoothing_configs(seasonal=[None]):
	models = list()
	# define config lists
	t_params = ['add', 'mul', None]
	d_params = [True, False]
	s_params = ['add', 'mul', None]
	p_params = seasonal
	b_params = [True, False]
	r_params = [True, False]
	# create config instances
	for t in t_params:
		for d in d_params:
			for s in s_params:
				for p in p_params:
					for b in b_params:
						for r in r_params:
							cfg = [t,d,s,p,b,r]
							models.append(cfg)
	return models
 

In [5]:
if __name__ == '__main__':
	# model configs
	cfg_list = exp_smoothing_configs(seasonal=[7,365])
	# grid search
	scores = grid_search(train_df[6], validation_df[6], cfg_list)
	print('done')
	# list top 3 configs
	for cfg, error in scores[:3]:
		print(cfg, error)

 > Model[['add', True, 'add', 7, False, False]] 9.659
 > Model[['add', True, 'add', 7, False, True]] 9.665
 > Model[['add', True, 'add', 365, False, True]] 51.646
 > Model[['add', True, 'add', 365, False, False]] 51.629
 > Model[['add', True, None, 7, False, True]] 40.212
 > Model[['add', True, None, 365, False, True]] 40.212
 > Model[['add', False, 'add', 7, False, True]] 10.056
 > Model[['add', False, 'add', 7, False, False]] 10.044
 > Model[['add', True, None, 7, False, False]] 40.211
 > Model[['add', True, None, 365, False, False]] 40.211
 > Model[['add', False, None, 7, False, True]] 41.170
 > Model[['add', False, None, 7, False, False]] 43.031
 > Model[['add', False, None, 365, False, True]] 41.170
 > Model[['add', False, None, 365, False, False]] 43.031
 > Model[['add', False, 'add', 365, False, True]] 52.897
 > Model[['add', False, 'add', 365, False, False]] 52.865
 > Model[[None, False, None, 7, False, False]] 40.144
 > Model[[None, False, None, 7, False, True]] 40.144
 > Mode

best Holt Winters parameters:
* [None, False, 'add', 7, False, True] 9.594741561630848
* [None, False, 'add', 7, False, False] 9.647501354037272
* ['add', True, 'add', 7, False, False] 9.658548315335132

In [6]:
def walk_forward_prediction(train,test, cfg):
	predictions = list()
	# seed history with training dataset
	history = [x for x in train]
	# step over each time-step in the test set
	for i in range(len(test)):
		# fit model and make forecast for history
		yhat = exp_smoothing_forecast(history, cfg)
		# store forecast in list of predictions
		predictions.append(yhat)
		# add actual observation to history for the next loop
		history.append(test[i])
	# estimate prediction error
	error = mape(test, predictions)
	return predictions[len(train):]

In [7]:
def residuals(y_true,y_predict):
    '''Returns list with residuals for all observations where y_true not 0. Where y_true is 0, the residuals are 0 as well '''
    return [y_true[idx] - y_predict[idx] if y_true[idx] != 0 else 0 for idx,_ in enumerate(y_true) ]

In [8]:
def pct_residuals(y_true,y_predict):
    '''Returns list with percentage errors for all observations where y_true not 0. Where y_true is 0, the percentage error is 0 as well'''
    return [(y_true[idx] - y_predict[idx])/y_true[idx] * 100 if y_true[idx] != 0 else 0 for idx,_ in enumerate(y_true) ]

In [9]:
# Plot cumulative density function of residuals
def residual_cdf(data):
    '''Plots cdf of residual input data'''
    # sort the data:
    data_sorted = np.sort(data)

    # calculate the proportional values of samples
    p = 1. * np.arange(len(data)) / (len(data) - 1)

    # plot the sorted data:
    fig = plt.figure(figsize=(20,15))


    ax1 = fig.add_subplot(311)
    ax1.plot(data_sorted, p)
    ax1.set_title('Residuals Cumulative Distribution Function')
    ax1.set_xlabel('Residuals');
    ax1.set_ylabel('Cumulative Distribution');
    ax1.axvline(x=np.percentile(data,5),color='r') 
    ax1.axvline(x=np.percentile(data,95),color='r')

    ax2 = fig.add_subplot(312)
    ax2.plot([idx for idx,_ in enumerate(data)],data,'bo');
    ax2.plot([idx for idx,_ in enumerate(data)],np.zeros(len(data)),'r-');
    ax2.set_title('Residuals over time')
    ax2.set_xlabel('Time in days');
    ax2.set_ylabel('Residual');  
    
    #Here, we could also add Q-Q plot and auto correlation plot for the residuals

In [10]:
def plot_prediction(y_true,y_predict):
    '''Plots true and predicted values on same y-axis'''
    fig = plt.figure(figsize=(20,15))
    ax1 = fig.add_subplot(311)
    ax1.plot(range(len(y_true)), y_true,'bo')
    ax1.plot(range(len(y_predict)),y_predict,'r-')
    ax1.set_title('Complete prediction')
    
    ax2 = fig.add_subplot(312)
    ax2.plot(range(len(y_true[:60])), y_true[:60],'bo')
    ax2.plot(range(len(y_predict[:60])),y_predict[:60],'r-o')
    ax2.set_title('Prediction first 60 days')
    ax2.set_ylim(0,max(y_true))
    ax3 = fig.add_subplot(313)
    ax3.plot(range(len(y_true[-60:])), y_true[-60:],'bo')
    ax3.plot(range(len(y_predict[-60:])),y_predict[-60:],'ro-')
    ax3.set_title('Prediction last 60 days')

In [11]:
def management_summary(y_true,y_predict):
    data = pd.DataFrame.from_dict({'y_true':y_true, 'y_predict':y_predict})
    
    #only regard data where y_true is not 0
    ex_0 = data[data['y_true'] != 0]
    
    #calculate how ofter we under- and over-estimate the revenue
    pct_lower = round(sum(ex_0.y_predict - ex_0.y_true < 0)/len(ex_0.y_true) * 100,1)
    pct_higher = round(100 - pct_lower,1)
    
    #calculate cumulative sums of under- and over estimation
    cumsum_lower = np.cumsum([np.abs(ex_0.y_predict[idx] - ex_0.y_true[idx]) if ex_0.y_predict[idx] < ex_0.y_true[idx] else 0 for idx,y in enumerate(ex_0.y_true) ])
    cumsum_higher = np.cumsum([np.abs(ex_0.y_predict[idx] - ex_0.y_true[idx]) if ex_0.y_predict[idx] > ex_0.y_true[idx] else 0 for idx,y in enumerate(ex_0.y_true)])

    
    fig = plt.figure(figsize=(20,15))
    ax1 = fig.add_subplot(211)
    ax1.plot(range(len(ex_0.y_true)), cumsum_lower,'b-o')
    ax1.plot(range(len(ex_0.y_true)),cumsum_higher,'r-o')
    
    ax1.set_title('Cumulative Sums of Under- and Over-Estimation')
    ax1.set_xlabel('Time')
    ax1.set_ylabel('Cumulated sum of errors')
    ax1.legend(['Under Estimation', 'Over Estimation', 'True Values'])
    
    ax2 = ax1.twinx()
    color = 'black'
    ax2.set_ylabel('Measured Values', color = color)
    ax2.plot(range(len(ex_0.y_true)),ex_0.y_true,'--', color=color, marker=10)
    
    return f'The model underestimates {pct_lower}% of the time'

In [12]:
#get prediction values for best model
prediction = walk_forward_prediction(train, test, scores[0])

#get errors of prediction
errors = residuals(test,prediction)

#get percentage errors of prediction
pct_errors = pct_residuals(test,prediction)

#

NameError: name 'train' is not defined